# EXAMEN 2

WelfareBnk es una fintech que incursionó hace cinco años en el mercado Mexicano (adquirió una
instituci´on financiera que ten´ıa cincuenta años en el pa´ıs). Actualmente, la institución tiene diversos
problemas relacionados con la cartera vencida y la pérdida de clientes, por lo que ha solicitado tu empresa
de consultoría el desarrollo de dos servicios/aplicaciones que le ayuden a:

Identificar clientes que potencialmente abandonar´an el banco. Esto les permitirá anticiparse y ofrecer
a los clientes incentivos para no abandonar la institución.
Noviembre 2024 ITESO, Universidad Jesuita de Guadalajara
Reducir la cartera vencida, mediante una mejor evaluación de riesgos sobre la solicitudes de créditos
realizadas por los clientes.

Como ingeniero con experiencia en aprendizaje automático se te ha asignado la construcción, entre-
namiento y validación de modelos que puedan ser utilizados como parte de los servicios que ayudarán
a la empresa a evaluar mejor los riesgos de cr´edito al consumidor y la identificaci´on de clientes que
potencialmente abandonarán la institución.

In [ ]:
# Firstly we must unzip the songs data and metadata
'''
import zipfile
with zipfile.ZipFile('./muestra_datos_WelfareBnk.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')
'''

In [165]:
import numpy as np
import pandas as pd

#Load the CSV file into a Pandas DataFrame
risk = './data/riesgo_crediticio.csv'
abandonment = './data/abandono_de_clientes.csv'
risk_d = pd.read_csv(risk, delimiter=',')
abandonment_d = pd.read_csv(abandonment, delimiter=',')
#print(risk_d.head())
#print(abandonment_d.head())

# Print the column names to debug
#print("Column names:", risk_d.columns)
#print("Column names:", abandonment_d.columns)

# Extract target column for each dataset
risk_Y = risk_d.pop('grado_credito')
abandonment_Y = abandonment_d.pop('abandono')

# Extract the features for each dataset
# For Risk, extract Historial crediticio, Incumplimiento histórico, Monto de crédito, Proporción de ingreso and Antigüedad en el empleo
risk_X = risk_d[['historial_crediticio', 'incumplimiento_historico', 'monto_credito', 'procentaje_de_ingreso', 'antiguedad_empleo']]
risk_X['incumplimiento_historico'] = risk_X['incumplimiento_historico'].replace(['Y', 'N'], [1, 0])
risk_Y.replace(['A', 'B', 'C', 'D', 'E', 'F', 'G'], [0, 1, 2, 3, 4, 5, 6], inplace=True)
# For Abandonment, extract Permanencia, Número de productos, Es cliente activo, Saldo, Puntuación crediticia
abandonment_X = abandonment_d[['permanencia', 'num_de_productos', 'es_cliente_activo', 'saldo', 'puntuacion_crediticia']]

# Check the shape of the data
print(risk_X.shape)
print(risk_Y.shape)

print(abandonment_X.shape)
print(abandonment_Y.shape)

# Check for balance in the target variable
#print(risk_Y.value_counts())
#print(abandonment_Y.value_counts())

# Count the number of missing values
#print(risk_X.isnull().sum())
#print(abandonment_X.isnull().sum())

for column in risk_X.columns:
    risk_X[column] = risk_X[column].fillna(0)

# Count the number of missing values
# print(risk_X.isnull().sum())
# print(abandonment_X.isnull().sum())

# Balance the target variable
from imblearn.under_sampling import ClusterCentroids

clustercentroids = ClusterCentroids()
risk_X, risk_Y = clustercentroids.fit_resample(risk_X, risk_Y)
abandonment_X, abandonment_Y = clustercentroids.fit_resample(abandonment_X, abandonment_Y)

# Check the shape of the data
print(risk_X.shape)
print(risk_Y.shape)

print(abandonment_X.shape)
print(abandonment_Y.shape)

# Check for balance in the target variable
print(risk_Y.value_counts())
print(abandonment_Y.value_counts())

C:\Users\Alex Casillas\AppData\Local\Temp\ipykernel_7868\3298153252.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  risk_X['incumplimiento_historico'] = risk_X['incumplimiento_historico'].replace(['Y', 'N'], [1, 0])
C:\Users\Alex Casillas\AppData\Local\Temp\ipykernel_7868\3298153252.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  risk_X['incumplimiento_historico'] = risk_X['incumplimiento_historico'].replace(['Y', 'N'], [1, 0])
C:\Users\Alex Casillas\AppData\Local\Temp\ipykernel_7868\3298153252.py:24: FutureW

(32581, 5)
(32581,)
(9960, 5)
(9960,)
(448, 5)
(448,)
(4058, 5)
(4058,)
grado_credito
0    64
1    64
2    64
3    64
4    64
5    64
6    64
Name: count, dtype: int64
abandono
0    2029
1    2029
Name: count, dtype: int64


In [166]:
#Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# For Risk
risk_X_train, risk_X_test, risk_Y_train, risk_Y_test = train_test_split(risk_X, risk_Y, test_size=0.2, random_state=42)

# For Abandonment
abandonment_X_train, abandonment_X_test, abandonment_Y_train, abandonment_Y_test = train_test_split(abandonment_X, abandonment_Y, test_size=0.2, random_state=42)

# Convert the DataFrame and the target column to TensorFlow tensors
import tensorflow as tf

# For Risk
risk_X_train = tf.constant(risk_X_train, dtype=tf.float32)
risk_X_test = tf.constant(risk_X_test, dtype=tf.float32)
risk_Y_train = tf.constant(risk_Y_train, dtype=tf.float32)
risk_Y_test = tf.constant(risk_Y_test, dtype=tf.float32)

# For Abandonment
abandonment_X_train = tf.constant(abandonment_X_train, dtype=tf.float32)
abandonment_X_test = tf.constant(abandonment_X_test, dtype=tf.float32)
abandonment_Y_train = tf.constant(abandonment_Y_train, dtype=tf.float32)
abandonment_Y_test = tf.constant(abandonment_Y_test, dtype=tf.float32)

# Verify the tensors
print(risk_X_train.shape, risk_X_test.shape, risk_Y_train.shape, risk_Y_test.shape) 
print(abandonment_X_train.shape, abandonment_X_test.shape, abandonment_Y_train.shape, abandonment_Y_test.shape)

(358, 5) (90, 5) (358,) (90,)
(3246, 5) (812, 5) (3246,) (812,)


In [167]:
# Now we create 2 Nerual Network models, one for each dataset, where we will
# use binary classification for the Abandonment dataset and multi-class classification for the Risk dataset

# For Risk
risk_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

risk_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# For Abandonment
abandonment_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

abandonment_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision'])



In [168]:
from sklearn.utils import class_weight

# Calculate class weights for the Risk dataset

# Calculate class weights for the Abandonment dataset
# abandonment_class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(abandonment_Y_train), y=abandonment_Y_train)
# abandonment_class_weights = dict(enumerate(abandonment_class_weights))

# Train the models

# For Risk
risk_model.fit(risk_X_train, risk_Y_train, epochs=0)

# For Abandonment
abandonment_model.fit(abandonment_X_train, abandonment_Y_train, epochs=100)

Epoch 1/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 449us/step - Precision: 0.5652 - accuracy: 0.5674 - loss: 474.2344
Epoch 2/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - Precision: 0.5811 - accuracy: 0.5696 - loss: 132.3090
Epoch 3/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - Precision: 0.5751 - accuracy: 0.5742 - loss: 88.9718
Epoch 4/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - Precision: 0.5867 - accuracy: 0.5688 - loss: 235.0844
Epoch 5/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - Precision: 0.5757 - accuracy: 0.5659 - loss: 87.6761
Epoch 6/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - Precision: 0.5948 - accuracy: 0.5782 - loss: 165.8832
Epoch 7/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - Precision: 0.5838 - accuracy: 0.5640 - loss: 167.1581
Epoch 8/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - Precision: 0.5935 - accuracy: 0.5799 - loss: 142.6151
Epoch 9/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - Precision: 0.5402 - accuracy: 0.5402 - loss: 76.8942
Epoc

In [169]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
# Test the models

# For Risk

risk_loss, risk_accuracy = risk_model.evaluate(risk_X_test, risk_Y_test)
print("Risk model accuracy:", risk_accuracy)

# For Abandonment

abandonment_Y_pred = abandonment_model.predict(abandonment_X_test)
# print("Abandonment model predictions:", abandonment_Y_pred)
# print("Abandonment Actual:", abandonment_Y_test)
# Round the predictions to 0 or 1
abandonment_Y_pred = np.round(abandonment_Y_pred)

# Confusion matrix for the Abandonment model
cm = confusion_matrix(abandonment_Y_test, abandonment_Y_pred)
print("Confusion matrix:")
print(cm)

# Convert tensors to numpy arrays for metric calculation
y_test_np = abandonment_Y_test.numpy()

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test_np, abandonment_Y_pred)
precision = precision_score(y_test_np, abandonment_Y_pred)
recall = recall_score(y_test_np, abandonment_Y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1214 - loss: 2581.5049  
Risk model accuracy: 0.13333334028720856
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step
Confusion matrix:
[[429   3]
 [295  85]]
Accuracy: 0.6330049261083743
Precision: 0.9659090909090909
Recall: 0.2236842105263158
